In [25]:
from  bfp import *
import mfem.ser as mfem
from bfp import coeff, assembler, solver, mesh, vis

In [ ]:
class QFuncCoefficient2(mfem.PyCoefficient):
    def __init__(self, pn, a_val=None, b_val=None, c_val=None, d_val=None, e_val=None, f_val=None, xs_t_val=0, mu_val=0, S_val=0, q_const=0):
        super(QFuncCoefficient2, self).__init__()
        self.pn = pn
        self.a_val = a_val
        self.b_val = b_val
        self.c_val = c_val
        self.d_val = d_val
        self.e_val = e_val
        self.f_val = f_val
        self.xs_t_val = xs_t_val
        self.mu_val = float(mu_val)
        self.S_val = S_val
        self.q_const = q_const

    def EvalValue(self, x):
        if self.pn == 3:
            return self.coeff_pn3(x)
        elif self.pn == 4:
            return self.coeff_pn4(x)
        elif self.pn == 5:
            return self.coeff_pn5(x)
        elif self.pn == 6:
            return self.coeff_pn6(x)
        elif self.pn == 7:
            return self.coeff_pn7(x)

    def coeff_pn3(self, x):
        return float(self.mu_val * self.b_val + self.xs_t_val * (self.a_val + self.b_val * x[0]))

    def coeff_pn4(self, x):
        return float(self.xs_t_val * (self.a_val + self.b_val * x[1]) + self.S_val * self.b_val)

    def coeff_pn5(self, x):
        sol =  self.mu_val * self.b_val * x[1] \
            + self.xs_t_val * (self.a_val + self.b_val * x[0] * x[1]) \
            + self.S_val * self.b_val * x[0]
        return sol
    
    def coeff_pn6(self, x):
        sol =  self.mu_val * ( 2 * self.b_val * x[0] + x[1])
        + self.xs_t_val * (self.a_val + self.b_val * x[0] ** 2 
                           + self.c_val * self.mu_val ** 2 + x[0] * x[1]) + self.S_val * x[0]
        return sol
    
    def coeff_pn7(self, x):
        sol = self.mu_val * (self.b_val + 2 * self.d_val * x[0] + self.f_val * self.mu_val) 
        + self.xs_t_val * (self.a_val + self.b_val * x[0] + self.c_val * self.mu_val + self.d_val * x[0] * 2 
                       + self.e_val * self.mu_val ** 2 + self.f_val * x[0] * self.mu_val)
        return sol
    
def Solve_Psi2(pn, mu_vals, w_vals, mesh, fes, xs_t_coeff, xs_t_const, S_const, 
              alpha, beta, dir_bdr1, dir_bdr2, a_val=None, b_val=None, c_val=None, d_val=None, e_val=None, 
              f_val=None, q_const=None, q_func=None, E_start=None, E_end=None, iter_=1000, tol=1e-12, p_level=1):

    psi_mu = []
    psi_mu_list = []

    for mu, w in zip(mu_vals, w_vals):
        a = a_val
        b = b_val
        c = c_val
        d = d_val
        e = e_val
        f = f_val
        inflow1 = inflow1_coeff
        inflow2 = b + d * mu ** 2
        print("  Solving for mu =", mu)
        inflow_coeff1 = coeff.InflowCoefficient(inflow1, mu)
        inflow_coeff2 = coeff.InflowCoefficient(inflow2, mu)
        v_coeff1 = coeff.VectorConstCoefficient([mu, 0.0])
        v_coeff2 = coeff.VectorConstCoefficient([0.0, S_const])
        if pn == 0:
            q_coeff = QFuncCoefficient2(pn, q_func)
        elif pn == 1 or pn == 2:
            q_coeff = QCoefficientE(q_const, E_start, E_end)
        elif pn == 3 or pn == 4 or pn == 5:
            q_coeff = QFuncCoefficient2(pn, a_val=a, b_val=b, xs_t_val=xs_t_const,
                                         mu_val=mu, S_val=S_const)
        elif pn == 6:
            q_coeff = QFuncCoefficient2(pn, a_val=a, b_val=b, c_val=c, d_val=d, e_val=e, f_val=f,
                                         xs_t_val=xs_t_const, mu_val=mu, S_val=S_const)

        a = mfem.BilinearForm(fes)
        a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff1, 1.0))
        a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff2, 1.0))
        a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
        a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff1, -alpha, beta)))
        a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff1, -alpha, beta)), dir_bdr1)
        a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff2, -alpha, beta)))
        a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff2, -alpha, beta)), dir_bdr2)
        a.Assemble()
        a.Finalize()
        A = a.SpMat()

        b = mfem.LinearForm(fes)
        b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
        b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff1, v_coeff1, -alpha, -beta), dir_bdr1)
        b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff2, -alpha, -beta), dir_bdr2)
        b.Assemble()

        psi = Initial_Guess(fes, 1.0)
        GMRES_solver(A, b, psi, iter_, tol, p_level)
        vis.GlVis_2D(mesh, psi)
        psi.Save("psi_hs_mu_{:.3f}.gf".format(mu))
        psi_mu.append(psi.GetDataArray())
        psi_mu_list.append((mu, w, psi))

    return psi_mu_list



In [48]:
# Set the problem parameters
nx = 10
nE = 10
x_start = 0.0
x_end = 1.0
E_start = 0.0
E_end = 1.0
N_ang = 2
order = 1
alpha = 1.0
beta = 0.5
ref_level = 2

# Problem 5: ψ = a+bxE
a = 5
b = 10
c = 3
d = 0
e = 0
f = 0
pn=6
S_const = 2.0
xs_t_const = 5.0
q_const = 0.0

#Set solver parameters
iter_ = 1000
tol = 1e-12
p_level = 0 #this is print level, 1 -> verbose, 0 -> silent

# Create mesh and set uniform refinement
mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)

# Define finite element space
fes = FESpace(order, mesh)


# Define boundary attributes
dir_bdr1, dir_bdr2 = BoundaryConditions(mesh, x_min=1, x_max=1, y_min=1, y_max=1)

# Set coefficients and Sn quadratures
S_coeff = StoppingPowerCoefficientE(S_const, E_start, E_end)
xs_t_coeff = TotalXSCoefficientE(xs_t_const, E_start, E_end)
mu_vals, w_vals = gauss_legendre_dirs(N_ang)

# Solution
psi_mu_pos_list = Solve_Psi2(pn, mu_vals, w_vals, mesh, fes, xs_t_coeff, xs_t_const, S_const, 
              alpha, beta, dir_bdr1, dir_bdr2, a_val=a, b_val=b, c_val=c, d_val=d, e_val=e, 
              f_val=f, q_const=q_const, E_start=E_start, E_end=E_end, iter_=1000, tol=1e-12, p_level=1)
phi = Solve_Phi(fes, psi_mu_pos_list)

File '/Users/melekderman/github/BFP/final/mesh/usr/10x10_2D.mesh' already exists.
  Solving for mu = -0.5773502691896257


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.5773502691896257


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…